# OmniSafe Tutorial - Environment Customization

OmniSafe: https://github.com/PKU-Alignment/omnisafe

Documentation: https://omnisafe.readthedocs.io/en/latest/

Gymnasium: https://github.com/Farama-Foundation/Gymnasium

[Gymnasium](https://github.com/Farama-Foundation/Gymnasium) is an open source Python library for developing and comparing reinforcement learning algorithms by providing a standard API to communicate between learning algorithms and environments, as well as a standard set of environments compliant with that API.

## 引言

在本节当中，我们将为您介绍如何将一个来自社区的已有环境嵌入OmniSafe中。[Gymnasium](https://github.com/Farama-Foundation/Gymnasium)提供的系列任务已被广泛应用至强化学习中。具体而言，本节将以[Pendulum-v1](https://gymnasium.farama.org/environments/classic_control/pendulum/)为例，展示如何将Gymnasium的任务嵌入OmniSafe。

## 快速安装

In [ ]:
# 通过pip安装（如果您已经安装，请忽略此段代码）
%pip install omnisafe

In [ ]:
# 通过源代码安装（如果您已经安装，请忽略此段代码）
## 克隆仓库
%git clone https://github.com/PKU-Alignment/omnisafe
%cd omnisafe

## 完成安装
%pip install -e .

## Gymnasium任务嵌入
环境嵌入需要的核心是为SafeRL智能体交互与训练提供足够的静态或动态信息，本节将详细介绍嵌入环境所必须定义的变量以及相应规范。我们将首先按照编写代码的逻辑顺序地展示整个嵌入过程，让您有一个初步的了解。然后我们将回顾所有代码，总结并整理您在自定义环境时需要进行的适配。


### 快速开始
首先，导入本教程所需要的所有外部变量。

In [ ]:
# 导入必要的包
from __future__ import annotations

from typing import Any, ClassVar
import gymnasium
import torch
import numpy as np
import omnisafe

from omnisafe.envs.core import CMDP, env_register, remove_from_register
from omnisafe.typing import DEVICE_CPU

随后，创建一个名为`ExampleMuJoCoEnv`的类，它需要继承的父类是`CMDP`。

In [ ]:
class ExampleMuJoCoEnv(CMDP):
    _support_envs: ClassVar[list[str]] = ['Pendulum-v1']  # 支持的任务名称

    need_auto_reset_wrapper = True  # 是否需要 `AutoReset` Wrapper
    need_time_limit_wrapper = True  # 是否需要 `TimeLimit` Wrapper

    def __init__(
        self,
        env_id: str,
        num_envs: int = 1,
        device: torch.device = DEVICE_CPU,
        **kwargs: Any,
    ) -> None:
        super().__init__(env_id)
        self._num_envs = num_envs
        self._env = gymnasium.make(id=env_id, autoreset=True, **kwargs)  # 实例化环境对象
        self._action_space = self._env.action_space  # 指定动作空间，以供算法层初始化读取
        self._observation_space = self._env.observation_space  # 指定观测空间，以供算法层初始化读取
        self._device = device  # 可选项，使用GPU加速。默认为CPU

    def reset(
        self,
        seed: int | None = None,
        options: dict[str, Any] | None = None,
    ) -> tuple[torch.Tensor, dict[str, Any]]:
        obs, info = self._env.reset(seed=seed, options=options)  # 重置环境
        return (
            torch.as_tensor(obs, dtype=torch.float32, device=self._device),
            info,
        )  # 将重置后的观测转换为torch tensor。

    @property
    def max_episode_steps(self) -> int | None:
        return self._env.env.spec.max_episode_steps  # 返回环境每一幕的最大交互步数

    def set_seed(self, seed: int) -> None:
        self.reset(seed=seed)  # 设定环境的随机种子以实现可复现性

    def render(self) -> Any:
        return self._env.render()  # 返回环境渲染的图像

    def close(self) -> None:
        self._env.close()  # 训练结束后，释放环境实例

    def step(
        self,
        action: torch.Tensor,
    ) -> tuple[
        torch.Tensor,
        torch.Tensor,
        torch.Tensor,
        torch.Tensor,
        torch.Tensor,
        dict[str, Any],
    ]:
        obs, reward, terminated, truncated, info = self._env.step(
            action.detach().cpu().numpy(),
        )  # 读取与环境交互后的动态信息
        cost = np.zeros_like(reward)  # Gymnasium并显式包含安全约束，此处仅为占位。
        obs, reward, cost, terminated, truncated = (
            torch.as_tensor(x, dtype=torch.float32, device=self._device)
            for x in (obs, reward, cost, terminated, truncated)
        )  # 将动态信息转换为torch tensor。
        if 'final_observation' in info:
            info['final_observation'] = np.array(
                [
                    array if array is not None else np.zeros(obs.shape[-1])
                    for array in info['final_observation']
                ],
            )
            info['final_observation'] = torch.as_tensor(
                info['final_observation'],
                dtype=torch.float32,
                device=self._device,
            )  # 将info中记录的上一幕final observation转换为torch tensor。

        return obs, reward, cost, terminated, truncated, info

有关上述代码的具体含义，我们已提供了详细的注释说明。更详细的解释可参考[Tutorial 3](./3.Environment%20Customization.ipynb)。我们将要点总结如下：

- **OmniSafe初始化需要的静态变量**

| 静态信息 | 必须 | 定义 | 类型 | 例子 |
|:---:|:---:|:---:|:---:|:---:|
| `need_auto_reset_wrapper` | 是 | 是否需要 `AutoReset` Wrapper | `bool`变量 | `True` |
| `need_time_limit_wrapper` | 是 | 是否需要 `TimeLimit` Wrapper | `bool`变量 | `True` |
| `_action_space` | 是 | 动作空间 | `gymnasium.space.Box` | `Box(low=-1.0, high=1.0, shape=(2,)` |
| `_observation_space` | 是 | 观测空间 | `gymnasium.space.Box` | `Box(low=-1.0, high=1.0, shape=(3,)` |
| `max_episode_steps` | 是 | 环境每一幕的最大交互步数 | 带有`@property`装饰器的，返回值为`int`或`None`类型变量的函数 | 参考上方代码块 |
| `_num_envs` | 否 | 并行环境数 | `int`变量 | 5 |
| `_device` | 否 | torch计算设备 | `torch.device`变量 | `DEVICE_CPU` |

- **OmniSafe需要环境提供的动态变量**

OmniSafe的智能体主要通过`reset`和`step`函数与环境进行动态交互。您需要确保定制化环境的返回值类型、个数与顺序与上述例子一致，更具体地：

| 动态信息 | 类型 | 个数 | 顺序 |
|:---:|:---:|:---:|:---:|
| `step` | `tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, dict[str, Any]]` | 6 | `obs`, `reward`, `cost`, `terminated`, `truncated`, `info` |
| `reset` | `tuple[torch.Tensor, dict[str, Any]]` | 2 | `obs`, `info` |

- **注意事项**

1. 尽管`_num_envs`与`_device`并不是必须指定的，但也请您在`__init__`函数中保留这两个参数的输入接口。
2. `_num_envs`是实例化多个环境并行采样的高级参数，它表示实例化环境的数目。如果您的定制化环境同样支持并行数指定，请通过`_num_envs`指定，而不用再定义一个新的接口。

随后，将上述环境通过注册装饰器`@env_register`注册入OmniSafe中，即可完成训练。

In [ ]:
@env_register
@remove_from_register  # 避免重复运行单元格时产生"环境已注册"报错
class ExampleMuJoCoEnv(ExampleMuJoCoEnv):
    pass


custom_cfgs = {
    'train_cfgs': {
        'total_steps': 200,
    },
    'algo_cfgs': {
        'steps_per_epoch': 200,
        'update_iters': 1,
    },
}
agent = omnisafe.Agent('PPOLag', 'Pendulum-v1', custom_cfgs=custom_cfgs)
agent.learn()

### 高级使用
除了上述使用方式外，来自社区的环境还可以享受OmniSafe的环境特定参数指定以及信息记录的特性。我们将详细展示具体操作方式。

#### 特定参数指定

以`Pendulum-v1`为例，根据Gymnasium的官方文档，创建该任务时可指定一个特定参数为`g`，即重力加速度。我们首先来看看它的默认取值：

In [ ]:
@env_register
@remove_from_register  # 避免重复运行单元格时产生"环境已注册"报错
class ExampleMuJoCoEnv(ExampleMuJoCoEnv):
    def __getattr__(self, name: str) -> Any:
        """Get the attribute of the environment."""
        if name.startswith('_'):
            raise AttributeError(f'attempted to get missing private attribute {name}')
        return getattr(self._env, name)


custom_cfgs = {
    'train_cfgs': {
        'total_steps': 200,
    },
    'algo_cfgs': {
        'steps_per_epoch': 200,
        'update_iters': 1,
    },
}
agent = omnisafe.Agent('PPOLag', 'Pendulum-v1', custom_cfgs=custom_cfgs)
agent.agent._env._env.g

我们实现了一个名为`__get_attr__`的魔法函数，用于调用并查看当前实例化的环境中的特定参数。在本例中，我们发现重力加速度`g`的默认值是10.0

通过查阅Gymnasium的文档，该参数可以在调用`gymnasium.make`函数创建环境的过程中指定。OmniSafe是否支持定制化环境的特定参数传递呢？答案是肯定的，具体操作也非常简单：

In [ ]:
custom_cfgs.update({'env_cfgs': {'g': 9.8}})
agent = omnisafe.Agent('PPOLag', 'Pendulum-v1', custom_cfgs=custom_cfgs)
agent.agent._env._env.g

非常好！重力加速度取值被我们更改为了9.8。我们只需要对`env_cfgs`进行操作，将需要定制参数的键与值指定，即可实现环境的特定参数传递。

#### 信息记录

`Pendulum-v1`任务有许多特定的动态信息，我们将为您介绍如何通过OmniSafe的`Logger`记录这些信息。具体而言，我们将以每幕角速度`angular_velocity`的最大值以及累计值为例为您讲解。

In [ ]:
from omnisafe.common.logger import Logger


@env_register
@remove_from_register  # 避免重复运行单元格时产生"环境已注册"报错
class ExampleMuJoCoEnv(ExampleMuJoCoEnv):

    def __init__(self, env_id, num_envs, device, **kwargs):
        super().__init__(env_id, num_envs, device, **kwargs)
        self.env_spec_log = {
            'Env/Max_angular_velocity': 0.0,
            'Env/Cumulative_angular_velocity': 0.0,
        }  # 在构造函数中重申并指定

    def spec_log(self, logger: Logger) -> None:
        for key, value in self.env_spec_log.items():
            logger.store({key: value})
            self.env_spec_log[key] = 0.0

    def step(self, action):
        obs, reward, cost, terminated, truncated, info = super().step(action=action)
        angle = obs[-1].item()
        self.env_spec_log['Env/Max_angular_velocity'] = max(
            self.env_spec_log['Env/Max_angular_velocity'], angle
        )
        self.env_spec_log['Env/Cumulative_angular_velocity'] += angle
        return obs, reward, cost, terminated, truncated, info


agent = omnisafe.Agent('PPOLag', 'Pendulum-v1', custom_cfgs=custom_cfgs)
agent.learn()

太好了！我们成功地在`Logger`中记录了需要的环境特定信息。值得注意的是，在这一过程中我们并没有修改OmniSafe的任何源代码。

## 总结
我们在本节使用了Gymnasium的经典环境`Pendulum-v1`，为您介绍了将一个社区已有的环境嵌入OmniSafe中所需的必要接口适配与信息提供。我们希望这个教程对您的定制化环境嵌入过程有帮助。如果您想将自己的环境作为OmniSafe官方支持的环境之一，或者在定制化环境中遇到了困难，欢迎在[Issues](https://github.com/PKU-Alignment/omnisafe/issues)，[Pull Requests](https://github.com/PKU-Alignment/omnisafe/pulls)与[Discussions](https://github.com/PKU-Alignment/omnisafe/discussions)模块与我们沟通。